In [ ]:
# RAG pipeline in a single cell using LangChain for PDF and Gemini

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

import os

# 1. Load PDF and extract text
def load_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    return loader.load()

# 2. Split text into chunks
def split_docs(docs, chunk_size=400, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(docs)

# 3. Embed and store in vector DB
def create_vector_store(docs):
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore

# 4. Build RAG QA chain with Gemini
def build_qa_chain(vectorstore):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)  # Requires GOOGLE_API_KEY env variable
    retriever = vectorstore.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain

# 5. Main RAG pipeline
def rag_pdf_qa(pdf_path, question):
    docs = load_pdf(pdf_path)
    chunks = split_docs(docs)
    vectorstore = create_vector_store(chunks)
    qa_chain = build_qa_chain(vectorstore)
    answer = qa_chain.run(question)
    return answer

# Example usage:
pdf_path = r"C:\Users\admin\Desktop\multimodel-rag-Customer_support\RAG_text\file-sample_150kB.pdf"  # Replace with your PDF file path
question = "What is the main topic of this document?"


print(rag_pdf_qa(pdf_path, question))

This document appears to be composed primarily of "Lorem ipsum" placeholder text, which is used to demonstrate the visual form of a document without conveying any meaningful content. Therefore, it does not have a main topic.


In [18]:
### RAG with multiple pdf and gemini 


from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
import os

# 1. Load one or many PDFs and extract text
def load_pdf(pdf_paths):
    if isinstance(pdf_paths, str):  # single path
        pdf_paths = [pdf_paths]
    docs = []
    for path in pdf_paths:
        loader = PyPDFLoader(path)
        docs.extend(loader.load())
    return docs

# 2. Split text into chunks
def split_docs(docs, chunk_size=400, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    return splitter.split_documents(docs)



# # 3. Embed and store in vector DB
# def create_vector_store(docs):
#     embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
#     vectorstore = FAISS.from_documents(docs, embeddings)
#     return vectorstore


### Embedding with GEMINI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
def create_vector_store(docs):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
    vectorstore = FAISS.from_documents(docs, embeddings)

    return vectorstore







from langchain.prompts import PromptTemplate
def build_qa_chain(vectorstore):

    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template="You are a helpful assistant. Use the following context to answer the question.\n\nContext: {context}\n\nQuestion: {question}\nAnswer:"
    )

    if not os.getenv("GOOGLE_API_KEY"):
        raise ValueError("GOOGLE_API_KEY environment variable is not set.")

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt_template}
    )
    return qa_chain






# # 4. Build RAG QA chain with Gemini
# def build_qa_chain(vectorstore):
#     if not os.getenv("GOOGLE_API_KEY"):
#         raise ValueError("GOOGLE_API_KEY environment variable is not set.")
    
#     llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)
#     retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
#     qa_chain = RetrievalQA.from_chain_type(
#         llm=llm,
#         retriever=retriever,
#         chain_type="stuff",
#         return_source_documents=True
#     )

#     return qa_chain




# 5. Main RAG pipeline
def rag_pdf_qa(pdf_paths, question):
    try:
        docs = load_pdf(pdf_paths)

        chunks = split_docs(docs)

        vectorstore = create_vector_store(chunks)


        qa_chain = build_qa_chain(vectorstore)

        result = qa_chain.invoke({"query": question})
   
        answer = result["result"]
        # sources = [doc.metadata for doc in result.get("source_documents", [])]
        return answer, sources
    except Exception as e:
        return f"Error while processing: {e}", []

# Example usage:
pdf_path = r"C:\Users\admin\Desktop\multimodel-rag-Customer_support\RAG_text\file-sample_150kB.pdf"
question = "What is the main topic of this document?"

answer, sources = rag_pdf_qa(pdf_path, question)

print("\nAnswer:\n", answer)
print("\nSources:\n", sources)



Answer:
 This document does not have a main topic. It is composed entirely of "Lorem ipsum" placeholder text, which is used to demonstrate the visual form of a document without conveying any meaningful content.

Sources:
 []


In [35]:
## LANgChain with webpage

import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 1. Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
all_splits = text_splitter.split_documents(docs)

# 2. Embed and store in vector DB
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(all_splits, embeddings)

# 3. Define a custom prompt (optional)
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="You are a precise assistant. Use ONLY the context to answer.If unsure, say 'I don't know'.\n\nContext: {context}\n\nQuestion: {question}\nAnswer:"
)

# 4. Build the RAG QA chain
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    #chain_type_kwargs={"prompt": prompt_template}
)

# 5. Ask a question
question = "Explain Self-Reflection"


result = qa_chain.invoke({"query": question})
print("Answer:\n", result["result"])






# # get retrieved chunks
# retrieved_docs = retriever.get_relevant_documents(question)

# print("\n=== Retrieved Chunks ===\n")
# for i, doc in enumerate(retrieved_docs, 1):
#     print(f"--- Chunk {i} ---")
#     print(doc.page_content[:500], "...")  # print first 500 chars for readability
#     print(f"Source: {doc.metadata}\n")

# # run QA chain
# result = qa_chain.invoke({"query": question})

# print("\n=== Answer ===\n")
# print(result["result"])









Answer:
 Self-reflection is a vital mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.

Here's how it works and its purpose:
*   **Purpose:** It enables agents to do self-criticism over past actions, learn from mistakes, and refine them for future steps, thereby improving the quality of final results and enhancing reasoning skills.
*   **Creation:** It is created by showing two-shot examples to a Large Language Model (LLM). Each example is a pair consisting of a failed trajectory and an ideal reflection designed to guide future changes in the agent's plan.
*   **Usage:** These generated reflections are then added into the agent’s working memory, typically up to three, to be used as context for querying the LLM in subsequent steps.
*   **Impact:** Based on the self-reflection results, an agent might compute a heuristic and opti